# Tensors

From the [programmer's guide](https://www.tensorflow.org/programmers_guide/tensors):

> TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
m1 = [[1.0, 2.0],
      [3.0, 4.0]]
      
m2 = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)
               
m3 = tf.constant([[1.0, 2.0],
     [3.0, 4.0]])

In [ ]:
print(type(m1))
print(type(m2))
print(type(m3))

In [ ]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

In [ ]:
print(type(t1))
print(type(t2))
print(type(t3))

Each tensor has a data type and a shape

In [ ]:
t = tf.constant([[1.0, 2.0, 3.0],
     [3.0, 4.0, 5.0]])
print(t.dtype)
print(t.get_shape())